<a href="https://colab.research.google.com/github/Yi-Wei-Lin/Tibame_ML_20210602/blob/main/DL_Imdb.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# urllib 方法
from urllib.request import urlretrieve
url = "http://ai.stanford.edu/~amaas/data/sentiment/aclImdb_v1.tar.gz"
urlretrieve(url, "IMDB.tar.gz")

#另一種方法 !wget
!wget 'http://ai.stanford.edu/~amaas/data/sentiment/aclImdb_v1.tar.gz' 

# 類似解壓縮的檔名
!tar xvzf IMDB.tar.gz

In [49]:
import glob
import pandas as pd
df = pd.DataFrame(columns=["content", "sentiment"])
for fn in glob.glob("aclImdb/train/pos/*.txt"):
  f = open(fn, "r", encoding="utf-8")
  content = f.read()
  f.close()
  s = pd.Series([content, 1], index=["content", "sentiment"])
  df = df.append(s, ignore_index=True)
for fn in glob.glob("aclImdb/train/neg/*.txt"):
  f = open(fn, "r", encoding="utf-8")
  content = f.read()
  f.close()
  s = pd.Series([content, 0], index=["content", "sentiment"])
  df = df.append(s, ignore_index=True)
df

,content,sentiment
0,This movie is excellent. I found it very inter...,1
1,this is one amazing movie!!!!! you have to rea...,1
2,Raggedy Ann and Raggedy Andy THE MOVIE is abou...,1
3,Master cinéaste Alain Resnais likes to work wi...,1
4,I love this film. It is well written and acted...,1
...,...,...
24995,First: I bought it at the video store. Second:...,0
24996,"This movie starts slow, then tapers off. After...",0
24997,This has to be one of the worst films of the 1...,0
24998,When I was little my parents took me along to ...,0


In [50]:
testdf = pd.DataFrame(columns=["content", "sentiment"])
for fn in glob.glob("aclImdb/test/pos/*.txt"):
  f = open(fn, "r", encoding="utf-8")
  content = f.read()
  f.close()
  s = pd.Series([content, 1], index=["content", "sentiment"])
  testdf = testdf.append(s, ignore_index=True)
for fn in glob.glob("aclImdb/test/neg/*.txt"):
  f = open(fn, "r", encoding="utf-8")
  content = f.read()
  f.close()
  s = pd.Series([content, 0], index=["content", "sentiment"])
  testdf = testdf.append(s, ignore_index=True)
testdf

,content,sentiment
0,This is a charming movie starring everyone's f...,1
1,I never dreamed when I started watching this D...,1
2,This is a little film with a big heart. Excell...,1
3,As others that have commented around the web.....,1
4,We saw La Spettatrice at the Syracuse Internat...,1
...,...,...
24995,"Set in 1945, Skenbart follows a failed Swedish...",0
24996,"I've seen every episode, and the characters ha...",0
24997,I've always loved horror flicks. From some of ...,0
24998,This mess starts off with a real tank running ...,0


In [52]:
from keras_preprocessing.text import Tokenizer
# 先創造你care的詞的詞典, 1000頁的的詞典(等一下每個詞都是1000維度的向量)
tok = Tokenizer(num_words=1000)
tok.fit_on_texts(df["content"])

In [ ]:
# 到底1000個詞是哪一個1000個詞(抓出次數最多的)
tok.word_index # 都是1000維度

In [64]:
# 把每一篇文章的每一個詞都轉化成為數字(1000維度的向量)
train_seq = tok.texts_to_sequences(df["content"])
test_seq = tok.texts_to_sequences(testdf["content"])
# 因為每一篇文章不一樣多, 出現最多詞的文章1496個詞, 其他文章就會補上NaN
pd.DataFrame(train_seq)

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,...,1456,1457,1458,1459,1460,1461,1462,1463,1464,1465,1466,1467,1468,1469,1470,1471,1472,1473,1474,1475,1476,1477,1478,1479,1480,1481,1482,1483,1484,1485,1486,1487,1488,1489,1490,1491,1492,1493,1494,1495
0,11,17,6,318,10,255,9,52,218.0,10.0,194.0,1.0,13.0,181.0,642.0,1.0,113.0,13.0,79.0,84.0,14.0,70.0,14.0,1.0,362.0,937.0,2.0,226.0,7.0,7.0,3.0,52.0,651.0,220.0,20.0,211.0,8.0,1.0,652.0,4.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,11,6,28,477,17,22,25,5,920.0,12.0,6.0,2.0,47.0,23.0,207.0,534.0,493.0,534.0,10.0,543.0,119.0,21.0,388.0,282.0,18.0,10.0,95.0,8.0,392.0,9.0,6.0,52.0,22.0,40.0,25.0,5.0,190.0,48.0,9.0,405.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,2,1,17,6,41,12,213,5,110.0,51.0,1.0,710.0,184.0,8.0,11.0,33.0,212.0,3.0,780.0,769.0,36.0,1.0,4.0,20.0,65.0,93.0,33.0,148.0,930.0,462.0,2.0,57.0,77.0,1.0,4.0,65.0,159.0,461.0,1.0,16.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,5,154,16,145,153,34,23,3,170.0,4.0,24.0,220.0,8.0,11.0,19.0,96.0,72.0,64.0,220.0,37.0,2.0,16.0,619.0,37.0,338.0,116.0,2.0,65.0,441.0,20.0,260.0,453.0,1.0,873.0,4.0,646.0,31.0,131.0,81.0,6.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,10,116,11,19,9,6,70,395,2.0,914.0,2.0,44.0,49.0,624.0,1.0,62.0,202.0,482.0,2.0,16.0,84.0,727.0,2.0,304.0,660.0,64.0,9.0,815.0,9.0,120.0,9.0,5.0,126.0,365.0,7.0,7.0,1.0,113.0,6.0,49.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
24995,83,10,9,30,1,371,330,10,293.0,9.0,837.0,9.0,13.0,355.0,9.0,13.0,21.0,160.0,88.0,4.0,1.0,68.0,832.0,2.0,233.0,18.0,21.0,219.0,42.0,21.0,61.0,3.0,75.0,17.0,42.0,3.0,959.0,7.0,7.0,10.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
24996,11,17,514,547,92,122,100,146,15.0,41.0,32.0,531.0,2.0,316.0,424.0,161.0,590.0,10.0,43.0,10.0,380.0,161.0,571.0,47.0,6.0,54.0,62.0,1.0,102.0,23.0,4.0,1.0,88.0,355.0,81.0,98.0,4.0,175.0,121.0,1.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
24997,11,44,5,27,28,4,1,246,105.0,4.0,1.0,51.0,58.0,365.0,10.0,68.0,146.0,11.0,19.0,109.0,1.0,308.0,9.0,13.0,30.0,72.0,40.0,293.0,1.0,83.0,317.0,32.0,531.0,16.0,260.0,1.0,30.0,86.0,75.0,9.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
24998,51,10,13,114,58,843,559,69,364.0,5.0,1.0,748.0,5.0,64.0,9.0,13.0,28.0,4.0,108.0,99.0,10.0,293.0,16.0,58.0,843.0,18.0,11.0,13.0,1.0,61.0,28.0,72.0,43.0,4.0,234.0,92.0,10.0,66.0,112.0,107.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [65]:
# 截長補短到每一篇文章都是同樣詞數
from keras.preprocessing.sequence import pad_sequences
train_pad = pad_sequences(train_seq, maxlen=100) # 擷取最後100個詞, 比較貼近人類情緒!?
test_pad = pad_sequences(test_seq, maxlen=100) 
pd.DataFrame(train_pad) # 0 就是補短

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,...,60,61,62,63,64,65,66,67,68,69,70,71,72,73,74,75,76,77,78,79,80,81,82,83,84,85,86,87,88,89,90,91,92,93,94,95,96,97,98,99
0,0,0,0,0,0,0,0,0,0,0,0,0,11,17,6,318,10,255,9,52,218,10,194,1,13,181,642,1,113,13,79,84,14,70,14,1,362,937,2,226,...,6,52,2,117,1,189,12,28,4,1,6,26,92,514,5,1,220,2,911,180,377,5,590,5,95,7,7,64,11,17,42,287,9,5,1,174,2,104,93,53
1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,5,920,12,6,2,47,23,207,534,493,534,10,543,119,21,388,282,18,10,95,8,392,9,6,52,22,40,25,5,190,48,9,405,22,354,1,17,2,354,1
2,21,32,853,705,91,3,386,618,90,8,1,519,2,67,128,27,506,635,91,102,23,29,951,2,1,687,23,84,35,84,22,235,166,620,95,55,2,55,171,199,...,179,1,299,8,11,303,37,3,173,4,82,36,1,8,840,128,91,345,69,634,112,852,2,235,128,180,635,9,283,772,5,166,18,3,5,398,8,58,4,2
3,33,254,82,770,3,22,8,134,228,16,55,2,831,5,1,4,3,884,4,403,6,116,71,338,9,13,150,593,8,12,90,3,640,726,19,60,13,79,41,116,...,413,453,5,456,15,65,6,37,3,497,60,6,21,30,29,9,6,3,19,8,60,1,618,600,6,8,400,16,91,11,6,28,4,1,135,11,19,67,711,27
4,9,815,9,120,9,5,126,365,7,7,1,113,6,49,2,258,124,3,475,289,1,628,10,506,29,1,102,2,255,95,5,27,218,2,70,16,7,7,10,48,...,503,453,34,149,64,12,579,9,128,90,69,651,51,282,949,43,70,8,1,127,555,11,594,13,112,16,3,752,46,53,20,1,2,103,11,250,651,2,438,19
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
24995,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,83,10,9,30,1,371,330,10,293,9,837,9,13,355,9,...,3,959,7,7,10,241,3,663,334,463,11,2,19,10,9,85,9,13,24,83,17,57,45,22,23,3,663,334,89,146,11,17,302,40,190,1,371,3,2,9
24996,175,121,1,4,11,19,97,27,8,3,4,45,33,350,5,9,14,3,428,47,6,54,10,97,25,577,1,169,62,16,3,375,128,16,1,226,6,3,975,42,...,58,110,146,116,8,1,55,4,275,18,12,10,3,84,5,27,3,10,97,25,74,1,83,5,43,18,10,3,224,96,193,302,10,293,41,888,81,43,156,69
24997,1,308,9,13,30,72,40,293,1,83,317,32,531,16,260,1,30,86,75,9,63,13,1,357,4,1,55,313,331,8,1,40,643,661,5,254,82,39,80,65,...,323,20,18,20,19,9,269,37,54,28,8,1,19,44,3,48,6,167,20,592,113,592,10,188,76,637,86,11,6,5,103,604,620,32,531,3,224,4,126,110
24998,268,20,2,20,5,1,210,118,22,40,178,5,29,4,95,42,112,41,42,61,41,9,6,161,50,71,3,450,621,5,32,721,1,5,5,94,102,35,8,530,...,15,18,24,20,24,746,140,2,19,9,498,1,210,4,10,542,383,11,28,45,332,544,3,114,96,651,2,356,139,5,22,4,338,894,900,40,11,19,112,571


In [85]:
from keras.models import Sequential
from keras.layers import Dense, Flatten, Embedding, Dropout
model = Sequential()
model.add(Embedding(input_dim=1000, output_dim=32, input_length=100))
model.add(Flatten())
model.add(Dense(256, activation="relu"))
model.add(Dropout(0.25)) # 隨機 drop 神經元 (類似隨機森林)
model.add(Dense(1, activation="sigmoid"))
model.summary()

Model: "sequential_4"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_4 (Embedding)      (None, 100, 32)           32000     
_________________________________________________________________
flatten_4 (Flatten)          (None, 3200)              0         
_________________________________________________________________
dense_8 (Dense)              (None, 256)               819456    
_________________________________________________________________
dropout (Dropout)            (None, 256)               0         
_________________________________________________________________
dense_9 (Dense)              (None, 1)                 257       
Total params: 851,713
Trainable params: 851,713
Non-trainable params: 0
_________________________________________________________________


In [86]:
model.compile(loss="binary_crossentropy",
      optimizer="adam",
      metrics=["accuracy"])

In [87]:
import numpy as np
s = np.array(df["sentiment"].astype("float")).reshape(25000, 1) #異常錯誤的處理
model.fit(train_pad,
    s,
    batch_size=200,
    epochs=3,
    validation_split=0.1)

Epoch 1/3
113/113 [==============================] - 6s 36ms/step - loss: 0.6160 - accuracy: 0.6295 - val_loss: 0.4167 - val_accuracy: 0.8060
Epoch 2/3
113/113 [==============================] - 4s 33ms/step - loss: 0.3143 - accuracy: 0.8672 - val_loss: 0.4588 - val_accuracy: 0.7992
Epoch 3/3
113/113 [==============================] - 4s 34ms/step - loss: 0.2224 - accuracy: 0.9169 - val_loss: 0.4674 - val_accuracy: 0.8044


In [88]:
ss = np.array(testdf["sentiment"].astype("float")).reshape(25000, 1) #異常錯誤的處理
model.evaluate(test_pad, ss)

782/782 [==============================] - 3s 4ms/step - loss: 0.4236 - accuracy: 0.8189


[0.42357200384140015, 0.818880021572113]

In [89]:
# 儲存模型
model.save("sentiment.h5")

In [91]:
# 讀取模型(如果你要繼續訓練, 不用重新 compile)
from keras.models import load_model
l = load_model("sentiment.h5")
l.evaluate(test_pad, ss)

782/782 [==============================] - 3s 4ms/step - loss: 0.4236 - accuracy: 0.8189


[0.42357200384140015, 0.818880021572113]

In [98]:
# 你可以選擇中間層當作輸出
# model.layers[0].output
# model.input
from keras.models import Model
partial = Model(model.input, model.layers[0].output)
# [0]取出第一篇文章 100個詞(32維度向量)
partial.predict(test_pad)[0][0]

array([-0.00728079, -0.00108213,  0.01449054, -0.00507213,  0.00659759,
        0.01641758,  0.00327567, -0.00949548, -0.00515725,  0.00851019,
        0.01064254, -0.00927078,  0.01895639,  0.0318676 , -0.01420996,
       -0.03000237, -0.01805029,  0.01891001, -0.00066478, -0.00392777,
        0.01433933, -0.00270458, -0.013021  , -0.01876394,  0.02829585,
        0.00892029,  0.00112664, -0.01254668,  0.00539812, -0.00048925,
       -0.02664521, -0.00683044], dtype=float32)